In [ ]:
from src.plot_multilayernetwork import *
from src.modules import *
import joblib
print("Loading network...")
network = joblib.load('networks/simple_all_new.pkl')
# 修改采样参数, 我们在这里保证不存在孤立节点
layers, interlayer_edges = prepare_sampled_network(
    network,
    w_media_ratio=1.0,  # 增加自媒体采样比例
    o_people_ratio=1.0,  # 增加普通用户采样比例
    min_degree=1,       # 降低最小度数要求
)

Loading network...

Network Statistics:
--------------------------------------------------

Mainstream Media Layer:
Nodes: 180
Edges: 95
Average Degree: 1.06

We Media Layer:
Nodes: 3246
Edges: 3296
Average Degree: 2.03

Public Layer:
Nodes: 2442
Edges: 2376
Average Degree: 1.95

Interlayer Connections: 1106


In [22]:
import networkx as nx
import numpy as np
from collections import Counter, defaultdict
import random
from scipy.optimize import curve_fit
import community.community_louvain as community_louvain
import matplotlib.pyplot as plt

def analyze_network_structure(layers, interlayer_edges):
    """Analyze structural features of multilayer networks with enhanced metrics"""
    layer_names = ['Mainstream Media', 'We Media', 'Public']
    results = {}
    
    # 1. Layer-specific analysis
    for i, (layer, name) in enumerate(zip(layers, layer_names)):
        print(f"\n{name} Layer Analysis:")
        print("-" * 50)
        
        metrics = {}
        n_nodes = layer.number_of_nodes()
        n_edges = layer.number_of_edges()
        
        # Basic topology
        print(f"Basic Metrics:")
        print(f"- Nodes: {n_nodes}")
        print(f"- Edges: {n_edges}")
        print(f"- Density: {nx.density(layer):.4f}")
        
        # Degree distribution analysis
        degrees = [d for n, d in layer.degree()]
        if len(degrees) > 5:
            degree_metrics = fit_truncated_powerlaw(degrees)
            print(f"\nDegree Distribution Analysis:")
            if 'error' not in degree_metrics:
                print(f"- Truncated Power-law α: {degree_metrics['alpha']:.3f}")
                print(f"- Exponential cutoff λ: {degree_metrics['lambda']:.3f}")
                print(f"- KS statistic: {degree_metrics['ks_statistic']:.3f}")
                print(f"- p-value: {degree_metrics['p_value']:.3f}")
                print(f"- Mean degree: {degree_metrics['mean_degree']:.3f}")
                print(f"- Std degree: {degree_metrics['std_degree']:.3f}")
            else:
                print(f"- Fitting failed: {degree_metrics['error']}")
                print(f"- Mean degree: {degree_metrics['mean_degree']:.3f}")
                print(f"- Std degree: {degree_metrics['std_degree']:.3f}")
            metrics['degree_dist'] = degree_metrics
        
        # Community structure
        if n_nodes > 0:
            community_metrics = analyze_communities(layer)
            print(f"\nCommunity Analysis:")
            print(f"- Modularity: {community_metrics['modularity']:.3f}")
            print(f"- Number of communities: {community_metrics['num_communities']}")
            metrics['community'] = community_metrics
        
        # Network robustness
        robustness_metrics = analyze_network_robustness(layer)
        print(f"\nRobustness Analysis:")
        print(f"- Robustness score: {robustness_metrics['robustness_score']:.3f}")
        metrics['robustness'] = robustness_metrics
        
        results[name] = metrics
    
    # 2. Interlayer analysis
    print("\nInterlayer Connection Analysis:")
    print("-" * 50)
    layer_connections = defaultdict(int)
    for l1, n1, l2, n2 in interlayer_edges:
        layer_connections[(l1, l2)] += 1
    
    total_interlayer = len(interlayer_edges)
    for (l1, l2), count in layer_connections.items():
        percentage = (count/total_interlayer*100) if total_interlayer > 0 else 0
        print(f"{layer_names[l1]} -> {layer_names[l2]}: {count} connections ({percentage:.1f}%)")
    
    return results

def fit_truncated_powerlaw(degrees):
    """Improved power-law fitting with adaptive boundaries"""
    degrees = np.array(degrees)
    counts = Counter(degrees)
    x = np.array(sorted(counts.keys()))
    y = np.array([counts[k]/len(degrees) for k in x])
    
    # 根据数据特征自适应设置边界
    max_alpha = 6.0 if len(degrees) > 1000 else 3.0
    bounds = ([1.5, 0.001], [max_alpha, 1.0])
    
    def truncated_powerlaw(x, alpha, lambda_):
        valid_x = np.maximum(x, 1e-10)
        return valid_x**(-alpha) * np.exp(-lambda_ * valid_x)
    
    try:
        popt, pcov = curve_fit(truncated_powerlaw, x, y, 
                             p0=[2.5, 0.1],
                             bounds=bounds,
                             maxfev=5000)
        
        # 计算拟合优度和KS检验
        y_fit = truncated_powerlaw(x, *popt)
        from scipy import stats
        ks_statistic, p_value = stats.ks_2samp(y, y_fit)
        
        return {
            'alpha': popt[0],
            'lambda': popt[1],
            'ks_statistic': ks_statistic,
            'p_value': p_value,
            'mean_degree': np.mean(degrees),
            'std_degree': np.std(degrees),
            'x': x,
            'y': y,
            'y_fit': y_fit
        }
    except Exception as e:
        return {
            'mean_degree': np.mean(degrees),
            'std_degree': np.std(degrees),
            'error': str(e)
        }

def analyze_communities(G):
    """Analyze community structure using Louvain method"""
    communities = community_louvain.best_partition(G)
    modularity = community_louvain.modularity(communities, G)
    
    return {
        'modularity': modularity,
        'num_communities': len(set(communities.values())),
        'community_sizes': dict(Counter(communities.values())),
        'node_communities': communities
    }

def analyze_network_robustness(G):
    """Analyze network robustness through percolation analysis"""
    gcc_sizes = []
    nodes = list(G.nodes())
    n_nodes = len(nodes)
    
    for i in range(10):
        remove_frac = i/10
        n_remove = int(remove_frac * n_nodes)
        
        G_temp = G.copy()
        removed = random.sample(nodes, n_remove)
        G_temp.remove_nodes_from(removed)
        
        if G_temp.number_of_nodes() > 0:
            gcc_size = len(max(nx.connected_components(G_temp), key=len))
            gcc_sizes.append(gcc_size/n_nodes)
        else:
            gcc_sizes.append(0)
    
    return {
        'percolation_curve': gcc_sizes,
        'robustness_score': np.trapz(gcc_sizes)/10
    }

In [23]:
# Analyze networks
results = analyze_network_structure(layers, interlayer_edges)



Mainstream Media Layer Analysis:
--------------------------------------------------
Basic Metrics:
- Nodes: 180
- Edges: 95
- Density: 0.0059

Degree Distribution Analysis:
- Truncated Power-law α: 1.500
- Exponential cutoff λ: 0.088
- KS statistic: 0.429
- p-value: 0.575
- Mean degree: 1.056
- Std degree: 2.699

Community Analysis:
- Modularity: 0.829
- Number of communities: 89

Robustness Analysis:
- Robustness score: 0.051

We Media Layer Analysis:
--------------------------------------------------
Basic Metrics:
- Nodes: 3246
- Edges: 3296
- Density: 0.0006

Degree Distribution Analysis:
- Truncated Power-law α: 5.351
- Exponential cutoff λ: 0.033
- KS statistic: 0.733
- p-value: 0.000
- Mean degree: 2.031
- Std degree: 47.824

Community Analysis:
- Modularity: 0.311
- Number of communities: 42

Robustness Analysis:
- Robustness score: 0.406

Public Layer Analysis:
--------------------------------------------------
Basic Metrics:
- Nodes: 2442
- Edges: 2376
- Density: 0.0008

Deg